import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=480
cls_num=2
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Ham(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (109, 432)
first_test_shape= (105, 432)
classes_quantity= 2
tr_lbls=	 {1, 2}
Count_labels= [52 57]
max(train_feature_Altitude)= 8.0329
min(train_feature_Altitude)= -2.0538
first_train_sample=
 [ 1.         0.11403    0.10489    0.095754   0.086619   0.086619
  0.086619   0.077484   0.077484   0.077484   0.077484   0.077484
  0.077484   0.077484   0.077484   0.077484   0.077484   0.077484
  0.068348   0.068348   0.068348   0.068348   0.068348   0.068348
  0.068348   0.068348   0.068348   0.077484   0.077484   0.077484
  0.077484   0.086619   0.086619   0.095754   0.095754   0.10489
  0.11403    0.11403    0.12316    0.12316    0.1323     0.14143
  0.14143    0.15057    0.1597     0.1597     0.16884    0.17797
  0.19624    0.20538    0.22365    0.25106    0.30587    0.36068
  0.37895    0.37895    0.43376    0.49771    0.47031    0.45204
  0.70783    1.2742     1.5026     1.4843     1.3564     1.2742
  1.192      1.0184     0.91794    0.89054    0.85399    0.70783
  0.

In [3]:
np.shape(ecg)

(109, 481)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 52	2 57	
 max = 57


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (52, 481)
Up to class  2 train shape =  (109, 481)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 52 57]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  8.36492882199105
min magnitude class 1  =  -1.7775228921276347
after normalizing >>
max magnitude class 1  =  1.0620379590784341
min magnitude class 1  =  -0.9444358147514927

 cls 2  >> 
max magnitude class 2  =  9.427653983004678
min magnitude class 2  =  -1.4538411248842698
after normalizing >>
max magnitude class 2  =  1.2722760971998848
min magnitude class 2  =  -0.8804020884546584
each class and its windows =
1 51	2 54	
 max instance in classes of test windows = 54


In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.9444358147514927
max tst = 1.2722760971998848


In [14]:
len(wndws_test)

105

In [15]:
np.shape(wndws_test)

(105, 481)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int16)


<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(52, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (109, 480)
y_train =>  (109,)
X_test  =>  (105, 480)
y_test  =>  (105,)
X_valid  =>  (109, 480)
y_valid  =>  (109,)


In [22]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
2/2 [==============================] - 4s 667ms/step - loss: 4.7430 - accuracy: 0.4862 - val_loss: 1.1837 - val_accuracy: 0.4771
Epoch 2/1000
2/2 [==============================] - 0s 39ms/step - loss: 3.7410 - accuracy: 0.4771 - val_loss: 1.2796 - val_accuracy: 0.4771
Epoch 3/1000
2/2 [==============================] - 0s 36ms/step - loss: 0.8840 - accuracy: 0.4679 - val_loss: 1.0389 - val_accuracy: 0.4771
Epoch 4/1000
2/2 [==============================] - 0s 48ms/step - loss: 1.4232 - accuracy: 0.5413 - val_loss: 1.1303 - val_accuracy: 0.5229
Epoch 5/1000
2/2 [==============================] - 0s 53ms/step - loss: 0.7386 - accuracy: 0.5505 - val_loss: 1.0871 - val_accuracy: 0.5596
Epoch 6/1000
2/2 [==============================] - 0s 39ms/step - loss: 0.6805 - accuracy: 0.5780 - val_loss: 1.0567 - val_accuracy: 0.5229
Epoch 7/1000
2/2 [==============================] - 0s 39ms/step - loss: 0.7784 - accuracy: 0.5138 - val_loss: 1.0819 - val_accuracy: 0.4771
Epoch 8/1000

2/2 [==============================] - 0s 39ms/step - loss: 1.0887e-05 - accuracy: 1.0000 - val_loss: 0.6327 - val_accuracy: 0.5413
Epoch 59/1000
2/2 [==============================] - 0s 40ms/step - loss: 9.8612e-06 - accuracy: 1.0000 - val_loss: 0.6135 - val_accuracy: 0.7523
Epoch 60/1000
2/2 [==============================] - 0s 38ms/step - loss: 9.9705e-06 - accuracy: 1.0000 - val_loss: 0.6150 - val_accuracy: 0.6606
Epoch 61/1000
2/2 [==============================] - 0s 39ms/step - loss: 7.2683e-06 - accuracy: 1.0000 - val_loss: 0.6089 - val_accuracy: 0.7890
Epoch 62/1000
2/2 [==============================] - 0s 38ms/step - loss: 7.2880e-06 - accuracy: 1.0000 - val_loss: 0.6171 - val_accuracy: 0.6239
Epoch 63/1000
2/2 [==============================] - 0s 39ms/step - loss: 6.8888e-06 - accuracy: 1.0000 - val_loss: 0.6149 - val_accuracy: 0.6330
Epoch 64/1000
2/2 [==============================] - 0s 39ms/step - loss: 4.6469e-06 - accuracy: 1.0000 - val_loss: 0.6215 - val_accuracy:

In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

4/4 [==============================] - 0s 50ms/step
array([[43,  8],
       [32, 22]], dtype=int64)
              precision    recall  f1-score   support

           2       0.57      0.84      0.68        51
           3       0.73      0.41      0.52        54

    accuracy                           0.62       105
   macro avg       0.65      0.63      0.60       105
weighted avg       0.66      0.62      0.60       105



In [27]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
2/2 [==============================] - 2s 540ms/step - loss: 3.4230 - accuracy: 0.1651 - val_loss: 10.2020 - val_accuracy: 0.4771
Epoch 2/1000
2/2 [==============================] - 0s 45ms/step - loss: 107.7111 - accuracy: 0.4771 - val_loss: 1.2963 - val_accuracy: 0.4771
Epoch 3/1000
2/2 [==============================] - 0s 63ms/step - loss: 1.0766 - accuracy: 0.4771 - val_loss: 1.2551 - val_accuracy: 0.5229
Epoch 4/1000
2/2 [==============================] - 0s 45ms/step - loss: 0.7851 - accuracy: 0.6422 - val_loss: 1.1701 - val_accuracy: 0.5229
Epoch 5/1000
2/2 [==============================] - 0s 45ms/step - loss: 0.8561 - accuracy: 0.5138 - val_loss: 1.1715 - val_accuracy: 0.5229
Epoch 6/1000
2/2 [==============================] - 0s 46ms/step - loss: 0.6554 - accuracy: 0.6881 - val_loss: 1.1266 - val_accuracy: 0.4771
Epoch 7/1000
2/2 [==============================] - 0s 45ms/step - loss: 1.0926 - accuracy: 0.4495 - val_loss: 1.2173 - val_accuracy: 0.5229
Epoch 8/1

2/2 [==============================] - 0s 45ms/step - loss: 3.2310e-05 - accuracy: 1.0000 - val_loss: 0.6784 - val_accuracy: 0.5229
Epoch 59/1000
2/2 [==============================] - 0s 45ms/step - loss: 2.8912e-05 - accuracy: 1.0000 - val_loss: 0.6938 - val_accuracy: 0.5229
Epoch 60/1000
2/2 [==============================] - 0s 45ms/step - loss: 2.4032e-05 - accuracy: 1.0000 - val_loss: 0.6870 - val_accuracy: 0.5229
Epoch 61/1000
2/2 [==============================] - 0s 45ms/step - loss: 1.8942e-05 - accuracy: 1.0000 - val_loss: 0.6839 - val_accuracy: 0.5229
Epoch 62/1000
2/2 [==============================] - 0s 46ms/step - loss: 1.6639e-05 - accuracy: 1.0000 - val_loss: 0.6722 - val_accuracy: 0.5229
Epoch 63/1000
2/2 [==============================] - 0s 46ms/step - loss: 1.3598e-05 - accuracy: 1.0000 - val_loss: 0.6820 - val_accuracy: 0.5229
Epoch 64/1000
2/2 [==============================] - 0s 47ms/step - loss: 1.2141e-05 - accuracy: 1.0000 - val_loss: 0.6779 - val_accuracy:

2/2 [==============================] - 0s 45ms/step - loss: 1.2796e-07 - accuracy: 1.0000 - val_loss: 0.7082 - val_accuracy: 0.4862
Epoch 115/1000
2/2 [==============================] - 0s 46ms/step - loss: 8.3118e-08 - accuracy: 1.0000 - val_loss: 0.7098 - val_accuracy: 0.4862
Epoch 116/1000
2/2 [==============================] - 0s 45ms/step - loss: 8.2025e-08 - accuracy: 1.0000 - val_loss: 0.6926 - val_accuracy: 0.4954
Epoch 117/1000
2/2 [==============================] - 0s 45ms/step - loss: 6.7807e-08 - accuracy: 1.0000 - val_loss: 0.7071 - val_accuracy: 0.4862
Epoch 118/1000
2/2 [==============================] - 0s 46ms/step - loss: 8.0931e-08 - accuracy: 1.0000 - val_loss: 0.7193 - val_accuracy: 0.4862
Epoch 119/1000
2/2 [==============================] - 0s 45ms/step - loss: 4.8121e-08 - accuracy: 1.0000 - val_loss: 0.6924 - val_accuracy: 0.4954
Epoch 120/1000
2/2 [==============================] - 0s 46ms/step - loss: 5.2496e-08 - accuracy: 1.0000 - val_loss: 0.6792 - val_acc

In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

4/4 [==============================] - 0s 42ms/step
array([[32, 19],
       [19, 35]], dtype=int64)
              precision    recall  f1-score   support

           2       0.63      0.63      0.63        51
           3       0.65      0.65      0.65        54

    accuracy                           0.64       105
   macro avg       0.64      0.64      0.64       105
weighted avg       0.64      0.64      0.64       105

